In [1]:
import os
#set cuda device = 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
from typing import Optional, Tuple, Union

In [3]:
import sys
sys.path.append('..')

In [4]:
from dem.energies.lennardjones_energy import LennardJonesEnergy

In [9]:
lj55_dist=LennardJonesEnergy(
            dimensionality= 165,
            n_particles= 55,
            data_path= "../data/test_split_LJ55-1000-part1.npy",
            data_path_train= "../data/train_split_LJ55-1000-part1.npy",
            data_path_val= "../data/test_split_LJ55-1000-part1.npy",
            device='cuda',
            data_normalization_factor= 1.0,
            is_molecule= True)
test_set = lj55_dist.setup_test_set()

In [25]:
std_dev_list=[0.025,0.0125,0.01,0.0075,0.005,0.0025]
for std in std_dev_list:
    print('std',std)
    inital_samples=torch.randn_like(test_set,device='cuda')
    initial_energy=lj55_dist(inital_samples)
    acceptance_rate=[]
    for i in tqdm.tqdm(range(10000)):
        new_samples=inital_samples+torch.randn_like(inital_samples)*std
        new_energy=lj55_dist(new_samples)
        acceptances=torch.exp(new_energy-initial_energy)>torch.rand_like(new_energy)
        acceptance_rate.append((acceptances.sum()/len(new_samples)).cpu().numpy())
        inital_samples[acceptances]=new_samples[acceptances]
        initial_energy[acceptances]=new_energy[acceptances]
    acceptance_rate=acceptance_rate[2000:]
    print('acceptance rate',np.mean(acceptance_rate))

std 0.025


  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [04:29<00:00, 37.09it/s]


acceptance rate 0.0163484
std 0.0125


100%|██████████| 10000/10000 [04:29<00:00, 37.07it/s]


...

acceptance rate 0.37665465
std 0.005


100%|██████████| 10000/10000 [04:29<00:00, 37.11it/s]


acceptance rate 0.54589415
std 0.0025


100%|██████████| 10000/10000 [04:29<00:00, 37.13it/s]

acceptance rate 0.69217014
